In [10]:
import os
print(os.getcwd())

d:\hospital\hospital-crawling


In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

driver_path = r'd:\hospital\hospital-crawling\chromedriver.exe'
options = webdriver.ChromeOptions()
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

sigungu_numbers = ['720', '200', '290', '710', '140', '230', '170', '260', '110']
sigungu_info = {
    '720': '군위군',
    '200': '남구',
    '290': '달서구',
    '710': '달성군',
    '140': '동구',
    '230': '북구',
    '170': '서구',
    '260': '수성구',
    '110': '중구'
}

hospital_data = []

try:
    url = 'https://www.nhis.or.kr/nhis/healthin/retrieveExmdAdminSearch.do'
    driver.get(url)
    wait = WebDriverWait(driver, 10)

    # 시도 설정
    select_sido_element = driver.find_element(By.ID, "search_sido")
    select_sido_object = Select(select_sido_element)
    select_sido_object.select_by_value("27")
    time.sleep(1)

    select_sigungu_element = driver.find_element(By.ID, "search_sigungu")
    select_sigungu_object = Select(select_sigungu_element)

    for sigungu_number in sigungu_numbers:
        select_sigungu_object.select_by_value(sigungu_number)
        time.sleep(2)

        # 검진 유형 설정
        if sigungu_number == '720':
            labels = driver.find_elements(By.CSS_SELECTOR, "label.checkbox.col-gap-8")
            labels[6].click()
            labels[7].click()
            labels[9].click()
        time.sleep(1)

        searchBtn = driver.find_element(By.ID, 'btnSearch')
        searchBtn.click()
        time.sleep(5)

        # pagination 돌면서 병원명, 주소 추출
        while True:
            # 페이지 소스 열기
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            # 현재 페이지 위치
            pagination = soup.select_one('div.pagination div.page')
            if not pagination:
                break
            
            current_page = pagination.select_one('strong')
            next_page = None

            if current_page:
                current_page_num = int(current_page.text.strip())
                page_links = pagination.select('a[onclick^="goPage"]')

                hospital_list = soup.select('ul#hospitalList > li')

                for li in hospital_list:
                    # 병원명 추출
                    title_div = li.select_one('button > div.list')
                    title_text = None
                    if title_div:
                        texts = list(title_div.stripped_strings)
                        title_text = texts[1]
                        print('병원명: ', title_text)

                    # 주소 추출
                    address_i = li.select_one('li.section.flex-col.col-gap-8.items-center')
                    address_text = None
                    if address_i:
                        address_text = address_i.get_text(strip=True).strip('"')
                        print('주소: ', address_text)

                    # 데이터 저장
                    hospital_data.append({'구':sigungu_info[sigungu_number],'병원명': title_text, '병원장': '', '주소':address_text})

                    # 다음 페이지 찾기
                    for a in page_links:
                        page_num = int(a['title'].strip())
                        if page_num > current_page_num:
                            next_page = a
                            break
                            
                if next_page:
                    next_page_number = next_page['onclick'].split('goPage(')[1].split(')')[0]
                    driver.execute_script(f"goPage({next_page_number});")
                    time.sleep(4)
                else:
                    break

        closeBtn = driver.find_element(By.CLASS_NAME, "ico.x24.ico-close")
        closeBtn.click()    

finally:
    # driver.quit()
    if hospital_data:
        df = pd.DataFrame(hospital_data)
        filename = "대구_구역별_위암대장암유방암_검진병원목록.xlsx"
        df.to_excel(filename, index=False)
        print(f"{filename} 저장 완료, ({len(df)}개 병원)")

병원명:  현대내과의원
주소:  대구광역시 군위군 군위읍 중앙길 88
병원명:  깨끗한속내과의원
주소:  대구광역시 남구 대봉로 61 4층 (봉덕동)
병원명:  대구가톨릭대학교병원
주소:  대구광역시 남구 두류공원로17길 33 (대명동)
병원명:  드림종합병원
주소:  대구광역시 남구 대명로 153 (대명동)
병원명:  몸편한내과신경과의원
주소:  대구광역시 남구 대명로 205 1,2,4층 (대명동)
병원명:  문성병원
주소:  대구광역시 남구 성당로 168 문성병원 (대명동)
병원명:  바른위앤장내과의원
주소:  대구광역시 남구 대명로 86 3층 (대명동)
병원명:  박정모내과  의원
주소:  대구광역시 남구 대명로 26 다온빌딩 2층 (대명동)
병원명:  삼성내과의원
주소:  대구광역시 남구 명덕로 322 (이천동)
병원명:  영남대학교병원
주소:  대구광역시 남구 현충로 170 (대명동)
병원명:  통합의료진흥원 전인병원
주소:  대구광역시 남구 두류공원로 77 (대명동)
병원명:  힘센병원
주소:  대구광역시 남구 봉덕로 71 (봉덕동)
병원명:  365상쾌한속연합내과의원
주소:  대구광역시 달서구 이곡공원로 12 3층 (이곡동)
병원명:  경대케이속내과의원
주소:  대구광역시 달서구 장기로 217 5층 (본리동)
병원명:  경일신경과내과의원
주소:  대구광역시 달서구 달구벌대로 1814 (두류동)
병원명:  계명대학교동산병원
주소:  대구광역시 달서구 달구벌대로 1035 (신당동)
병원명:  나사렛연합내과의원
주소:  대구광역시 달서구 용산로 141 6층 (용산동)
병원명:  나사렛종합병원
주소:  대구광역시 달서구 월배로 97 나사렛병원 (진천동)
병원명:  다나연합내과의원
주소:  대구광역시 달서구 용산로24길 87 101, 205호 (감삼동)
병원명:  달서미즈맘병원
주소:  대구광역시 달서구 월배로 62 미즈맘병원 지하1,2-5,8층 (진천동)
병원명:  더편한내과의원
주소:  대구광역시 달서구 월배로 487-1 6~7층 (송현동)
병원명:  